In [1]:
import chromadb
import cx_Oracle
#import pymysql
import numpy as np
import pandas as pd
import json
import ast

#import fitz  # PyMuPDF
import pdfplumber

import re
import os 
import openai
import docx
import time
from tqdm import tqdm
from kss import split_sentences  
from konlpy.tag import Kkma
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification

from chromadb.utils import embedding_functions

from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.docstore.document import Document

In [5]:
def get_txt_paths(folder_path):
    pdf_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                pdf_paths.append(os.path.join(root, file))
    return pdf_paths

In [6]:
root_path = './data/adobe_pdf/json_to_txt'
# 전처리 한 데이터 : 표 / 이미지 및 불필요한 단락 제거 
txt_list = get_txt_paths(root_path)
txt_list

['./data/adobe_pdf/json_to_txt\\BNK투자증권_포스코퓨처엠_20240409_20240703.txt',
 './data/adobe_pdf/json_to_txt\\DS투자증권_포스코퓨처엠_20230428_20240703.txt',
 './data/adobe_pdf/json_to_txt\\교보증권_포스코퓨처엠_20240202_003670_20190031_342_20240703.txt',
 './data/adobe_pdf/json_to_txt\\상상인_기업분석_포스코퓨처엠_20240425_20240703.txt',
 './data/adobe_pdf/json_to_txt\\신한투자증권_포스코퓨처엠_240426_20240703.txt',
 './data/adobe_pdf/json_to_txt\\유안타증권_포스코퓨처엠_240507_20240703.txt',
 './data/adobe_pdf/json_to_txt\\키움증권_포스코퓨처엠_240426_20240703.txt',
 './data/adobe_pdf/json_to_txt\\포스코퓨처엠_IBK투자증권_20231025_20240703.txt',
 './data/adobe_pdf/json_to_txt\\포스코퓨처엠_대신증권_20231025_20240703.txt',
 './data/adobe_pdf/json_to_txt\\하나증권_포스코퓨처엠_20240426_20240703.txt']

In [7]:
loader = DirectoryLoader(root_path, glob="*.txt", loader_cls=TextLoader, loader_kwargs={"encoding": "utf-8"})
documents = loader.load()

len(documents)

10

In [8]:
documents[0]

Document(page_content='하반기에는 실적 개선 + 원재료 수직계열화 시작\n재고평가손 환입이 더해져 1Q 실적은 예상보다 양호할 전망 1Q 연결OP 355억원으로 예상보다 양호할 전망이다. 시장 성장 둔화와 ASP 하락으로 에너지소재의 본원 수익성은 낮을 것으로 예상되나, 23.4Q에 인식 된 양극재 재고평가손실 중 일부분이 환입되면서 헤드라인 손익은 예상을 상 회할 전망이다. 손실환입까지 반영한 양극재OP는 177억원 (OPM 2.6%), 음 극재는 소폭 적자 (OPM -1%)가 예상된다. 양극재 출하량은 qoq 개선되나 ASP 하락으로 매출액은 전분기 수준으로 예상되고, 음극재는 인조흑연 초기 생산비용 등으로 저조한 수익성이 예상된다. 기초소재는 유가 상승에 따른 화성사업 손익 개선 등으로 안정적인 실적을 기록할 것으로 예상된다.\n하반기에는 뚜렷한 실적 회복세 예상\n2Q까지는 yoy 실적 모멘텀이 제한적이나, 하반기에는 뚜렷한 회복세가 예상 된다 (연결OP 3Q 598억원, 4Q 705억원 예상). 리튬 가격 급락이 촉발한 양 극재ASP 하락이 하반기에는 진정될 것으로 예상되기 때문이다. 양극재 가격 은 리튬 가격에 2분기 후행해서 변동되는데, 리튬 가격이 연초에 바닥을 찍 은 후 2~3월 반등했기 때문에 2Q말~3Q부터는 양극재 가격이 안정화될 전 망이다. 하반기 금리 인하까지 현실화된다면, 고객사의 재고 리빌딩과 전기차 구매수요 회복이 맞물려 실적이 빠르게 반등할 수 있을 것으로 예상된다.\n투자의견 매수, 목표주가 41만원 유지\n23년 하반기부터 본격화된 양극재 가격 하락은 올해 상반기를 기점으로 마무 리될 전망이다. 더불어 그룹사를 통한 원재료 소싱의 수직계열화는, 하반기부 터 점진적으로 시작돼 2025년에 본격화되며 Peers 대비 양극재 사업의 근원 적 경쟁력이 높아질 전망이다. 실적과 주가에 부담이었던 ASP 하락의 부정적 영향이 마무리되는 시점으로, 매수 접근을 추천한다.\n투자등급 (기업 투자의견은 향후 6개월간 추천일

In [ ]:
# RecursiveCharacterTextSplitter 사용 : 필요에 따라 수정 
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=10, separators=[' '])
#texts = text_splitter.split_documents(documents)

#print(len(texts))

In [9]:
class TextSplitter:
    def __init__(self, stock, keywords, date):
        self.stock = stock
        self.keywords = keywords
        self.date = date

    def split_documents(self, documents, delimiters):
        split_texts = []
        for doc in documents:
            text = doc.page_content
            pieces, positions = self._split_text(text, delimiters)
            for piece, (start, end) in zip(pieces, positions):
                if piece.strip():  # 빈 문자열이나 공백만 있는 문자열 제외
                    new_metadata = doc.metadata.copy()
                    new_metadata.update({
                        'sdate': self.date,
                        'timestamp': self.convert_to_timestamp(self.date),
                        'ticket': self.stock,
                        'keyword': self.keywords,
                        'citation': f"{{'start': {start}, 'end': {end}}}", # vectordb 문자열만 인식, 불러올땐 변경 ast 함수 사용  ast.literal_eval()
                        'full_text': text
                    })
                    split_texts.append(Document(page_content=piece, metadata=new_metadata))
        return split_texts

    @staticmethod
    def convert_to_timestamp(date_str):
        datetime_object = datetime.strptime(date_str, "%Y-%m-%d")
        return int(datetime_object.timestamp())

    def _split_text(self, text, delimiters):
        pattern = '|'.join(map(re.escape, delimiters))
        parts = []
        positions = []
        start = 0
        for m in re.finditer(pattern, text):
            end = m.end()
            parts.append(text[start:end])  # 구분자 포함하여 추가
            positions.append((start, end - 1))
            start = end
        if start < len(text):
            parts.append(text[start:])
            positions.append((start, len(text) - 1))
        return parts, positions

class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

# LineSplitter 클래스 정의
class LineSplitter:
    def __init__(self, stock, keywords, date):
        self.stock = stock
        self.keywords = keywords
        self.date = date

    def split_documents(self, documents):
        split_texts = []
        for doc_index, doc in enumerate(documents, start=1):
            text = doc.page_content
            lines = doc.page_content.split('\n')
            for i, line in enumerate(lines, start=1):
                new_metadata = doc.metadata.copy()
                new_metadata.update({
                    'sdate': self.date,  # 날짜 추가
                    'timestamp': self.convert_to_timestamp(self.date),  # 날짜 추가
                    'ticket': self.stock,  # 종목 추가
                    'keyword': self.keywords,  # 키워드 추가
                    'citation': f"line_{i}",  # 단락 위치 추가
                    'full_text': text
                })
                split_texts.append(Document(page_content=line, metadata=new_metadata))
        return split_texts

    @staticmethod
    def convert_to_timestamp(date_str):
        datetime_object = datetime.strptime(date_str, "%Y-%m-%d")
        return int(datetime_object.timestamp())

    @staticmethod
    def timestamp_convert(documents):
        key_add = 'timestamp'
        for doc in documents:
            datetime_object = doc.metadata['sdate']
            if isinstance(datetime_object, str):
                datetime_object = datetime.strptime(datetime_object, "%Y-%m-%d")
            timestamp = int(datetime_object.timestamp())
            doc.metadata[key_add] = timestamp
            doc.metadata['sdate'] = datetime_object.strftime("%Y-%m-%d")  # datetime 객체를 str로 변환
        return documents

    @staticmethod
    def filter_documents_by_date_range(documents, start_timestamp, end_timestamp):
        filtered_docs = []
        for doc in documents:
            doc_timestamp = doc.metadata.get('timestamp')
            if doc_timestamp and start_timestamp <= doc_timestamp <= end_timestamp:
                filtered_docs.append(doc)
        return filtered_docs

In [10]:
stock_name = "포스코퓨처엠"  # 예시 종목명
keywords = "증권사 리포트"  # 예시 키워드
current_date = datetime.now().strftime("%Y-%m-%d")  # 현재 날짜


select = 'text' # 'line'

if select == 'text':
    # Splitter 인스턴스 생성
    splitter = TextSplitter(stock=stock_name, keywords=keywords, date=current_date)

    # 구분자 목록
    delimiters = ['. '] #, '\n']
    texts = splitter.split_documents(documents, delimiters)

if select == 'line':
    splitter = LineSplitter(stock=stock_name, keywords=keywords, date=current_date)
    # 문서 분할 실행
    texts = splitter.split_documents(documents)

In [11]:
texts[0].page_content

'하반기에는 실적 개선 + 원재료 수직계열화 시작\n재고평가손 환입이 더해져 1Q 실적은 예상보다 양호할 전망 1Q 연결OP 355억원으로 예상보다 양호할 전망이다. '

In [12]:
texts[0].metadata

{'source': 'data\\adobe_pdf\\json_to_txt\\BNK투자증권_포스코퓨처엠_20240409_20240703.txt',
 'sdate': '2024-07-08',
 'timestamp': 1720364400,
 'ticket': '포스코퓨처엠',
 'keyword': '증권사 리포트',
 'citation': "{'start': 0, 'end': 90}",
 'full_text': '하반기에는 실적 개선 + 원재료 수직계열화 시작\n재고평가손 환입이 더해져 1Q 실적은 예상보다 양호할 전망 1Q 연결OP 355억원으로 예상보다 양호할 전망이다. 시장 성장 둔화와 ASP 하락으로 에너지소재의 본원 수익성은 낮을 것으로 예상되나, 23.4Q에 인식 된 양극재 재고평가손실 중 일부분이 환입되면서 헤드라인 손익은 예상을 상 회할 전망이다. 손실환입까지 반영한 양극재OP는 177억원 (OPM 2.6%), 음 극재는 소폭 적자 (OPM -1%)가 예상된다. 양극재 출하량은 qoq 개선되나 ASP 하락으로 매출액은 전분기 수준으로 예상되고, 음극재는 인조흑연 초기 생산비용 등으로 저조한 수익성이 예상된다. 기초소재는 유가 상승에 따른 화성사업 손익 개선 등으로 안정적인 실적을 기록할 것으로 예상된다.\n하반기에는 뚜렷한 실적 회복세 예상\n2Q까지는 yoy 실적 모멘텀이 제한적이나, 하반기에는 뚜렷한 회복세가 예상 된다 (연결OP 3Q 598억원, 4Q 705억원 예상). 리튬 가격 급락이 촉발한 양 극재ASP 하락이 하반기에는 진정될 것으로 예상되기 때문이다. 양극재 가격 은 리튬 가격에 2분기 후행해서 변동되는데, 리튬 가격이 연초에 바닥을 찍 은 후 2~3월 반등했기 때문에 2Q말~3Q부터는 양극재 가격이 안정화될 전 망이다. 하반기 금리 인하까지 현실화된다면, 고객사의 재고 리빌딩과 전기차 구매수요 회복이 맞물려 실적이 빠르게 반등할 수 있을 것으로 예상된다.\n투자의견 매수, 목표주가 41만원 유지\n23년 하반기부터 본격화된 양극재 가격

In [13]:
texts[0].metadata['citation']

"{'start': 0, 'end': 90}"

In [14]:
# vectorDB 딕셔너리 구조지원안하므로 위와 같이 처리 
import ast

text_len = texts[0].metadata['citation']

print(ast.literal_eval(text_len))
start = ast.literal_eval(text_len)['start']
end = ast.literal_eval(text_len)['end']
# citation 용 
texts[0].metadata['full_text'][start:end+1]

{'start': 0, 'end': 90}


'하반기에는 실적 개선 + 원재료 수직계열화 시작\n재고평가손 환입이 더해져 1Q 실적은 예상보다 양호할 전망 1Q 연결OP 355억원으로 예상보다 양호할 전망이다. '

In [ ]:
### 컬렉션 파트 ### 

In [6]:
client = chromadb.PersistentClient()

In [7]:
collection_name = 'sample_vdb'

In [20]:
collections = client.list_collections()
collections

[Collection(name=sample_vdb)]

In [19]:
collection = client.create_collection(
    name= collection_name,
    metadata={
        "hnsw:space": "cosine",                # 코사인 유사도
        "hnsw:construction_ef": 100,           # HNSW 그래프에서 탐색할 이웃 수 (건설 시)
        "hnsw:M": 16,                          # 최대 이웃 연결 수
        "hnsw:search_ef": 10,                 # 검색할 때 탐색할 이웃 수
        "hnsw:num_threads": 1,                 # 사용할 스레드 수
        "hnsw:resize_factor": 1.2,               # 그래프 성장률
        "hnsw:batch_size": 100,                # 브루트포스 인덱스 크기
        "hnsw:sync_threshold": 1024            # 디스크에 기록할 임계값
    }
)

In [17]:
# 삭제 : 필요시 사용 
client.delete_collection(name=collection_name)

In [21]:
start_time = time.time()

documents = texts

ids = [str(i) for i in range(len(documents))]
page_contents = [doc.page_content for doc in documents]
metadatas = [doc.metadata for doc in documents]
embeddings = embedding_function(page_contents) 

# 실행할 코드 블록 끝 시간 기록
end_time = time.time()

# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")

Execution Time: 14.013217687606812 seconds


In [22]:
collection.add(ids=ids, embeddings=embeddings, metadatas=metadatas, documents=page_contents)

In [ ]:
## 로드 

In [8]:
collection = client.get_collection(name= collection_name, embedding_function=embedding_function)

In [ ]:
#collection.get()

In [10]:
# 단락 테스트 
query_text = 'N으로 시작하는 제품명이 뭐지?'
query_embedding = embedding_function(query_text)[0] # vdb 불러올때 임베딩 함수 지정안하면 사용
print(len(query_embedding))
query_text

1024


'N으로 시작하는 제품명이 뭐지?'

In [11]:
# Let's read the contents of the text file located at './data/query/질문.txt' and extract its contents into a list, with each line as a separate list element.

# Define the file path
file_path = './data/query/검색해야할질문리스트.txt'

# Read the file and extract lines into a list
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Strip newline characters from each line
lines = [line.strip() for line in lines]

query_list = lines[:-1]
query_list

['N86 제품의 앞으로의 전망은 어떠한가?',
 '양극재 사업 수익성은 전망이 어떠한가?',
 '음극재 사업 수익성은 전망이 어떠한가?',
 '양극재 가격이 안정화될 것이라면 그 근거는 무엇인가?']

In [12]:
query_text = query_list[3]
query_text

'양극재 가격이 안정화될 것이라면 그 근거는 무엇인가?'

In [13]:
# 날짜 검색용 
start_date = datetime(2024, 7, 6)
end_date = datetime(2024, 7, 10)

start_timestamp = int(start_date.timestamp())
end_timestamp = int(end_date.timestamp())

start_timestamp, end_timestamp

(1720191600, 1720537200)

In [14]:
results = collection.query(
    #query_embeddings=[query_embedding],
    query_texts=[query_text],
    where={
        "$and": [
            {"timestamp": {"$gt": start_timestamp}},
            {"timestamp": {"$lt": end_timestamp}},
            {"ticket": "포스코퓨처엠"}
        ]
    },
    n_results=100
)

#doc_n = list(set(results['documents'][0]))
doc_n = results['documents'][0]

In [16]:
doc_n = list(set(results['documents'][0]))

In [17]:
# reranking
sent_list = reranker_model(query_text, doc_n)

In [18]:
prompt_template = """마지막 질문에 답하려면 다음 문맥을 사용하세요. 답을 모르면 모른다고 말하고 답을 만들어내려고 하지 마세요.
당신은 투자 전문가입니다. 다음 보고서 내용의 일부인 문맥 3개를 기반으로 질문에 성실히 답변해주세요.

## 문맥 : 
{doc_1}

{doc_2}

{doc_3}

질문: {user_query}
도움이 되는 답변:"""


doc_1 = sent_list[0]
doc_2 = sent_list[1]
doc_3 = sent_list[2]

in_prompt = prompt_template.format(user_query = query_text, doc_1 = doc_1, doc_2 = doc_2,  doc_3 = doc_3 )
print(in_prompt)

마지막 질문에 답하려면 다음 문맥을 사용하세요. 답을 모르면 모른다고 말하고 답을 만들어내려고 하지 마세요.
당신은 투자 전문가입니다. 다음 보고서 내용의 일부인 문맥 3개를 기반으로 질문에 성실히 답변해주세요.

## 문맥 : 
양극재 가격 은 리튬 가격에 2분기 후행해서 변동되는데, 리튬 가격이 연초에 바닥을 찍 은 후 2~3월 반등했기 때문에 2Q말~3Q부터는 양극재 가격이 안정화될 전 망이다. 

리튬 가격 급락이 촉발한 양 극재ASP 하락이 하반기에는 진정될 것으로 예상되기 때문이다. 

최근 메탈 가격 하락에 따른 중장기 추정 양극재 ASP 하향 및 실적 변경에 따라 목표주가 조정
4분기까지 이어질 단기 실적 부진이 아쉬우나, 중장기 공급계약으로 바인딩된 물 량 비중이 높은 24년부터 안정적인 실적 성장 가능할 것으로 예상(누적수주합산 얼티엄셀향 35조, SDI향 40조, LGES향 30조). 

질문: 양극재 가격이 안정화될 것이라면 그 근거는 무엇인가?
도움이 되는 답변:


In [19]:
## 여기서 부터 추론 
import time
import warnings
import logging
warnings.filterwarnings("ignore")

# transformers의 로깅 설정을 조정하여 경고 메시지 억제
logging.getLogger("transformers").setLevel(logging.ERROR)


sys_msg = "당신은 인공지능 어시스턴트입니다. 묻는 말에 친절하고 정확하게 답변하세요."
messages = [
    {"role": "system", "content": sys_msg},
    ]

start = time.time()  # 시작 시간 저장

#user_input = input('[user msg] = ')
user_input = in_prompt

user_msg = {"role": "user", "content": user_input}
messages.append(user_msg)

chat = tokenizer.apply_chat_template(messages, tokenize=False)
prompt = chat

response = inference_output(prompt)
split_output = '<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
outputs = response.split(split_output)[-1]
try:
    end_token = '<|end_of_text|'
    outputs = outputs.replace(end_token)
except:
    pass

citation_prompt =  citation_extract_prompt_add_v000(results, doc_1, doc_2, doc_3)

outputs_ex = outputs + '\n\n' + citation_prompt

print('[AI bot msg] = ', outputs_ex) # outputs_ex
print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

[AI bot msg] =  양극재 가격이 안정화될 것이라면 그 근거는 리튬 가격이 연초에 바닥을 찍은 후 2~3월에 반등했기 때문에 2Q말~3Q부터는 양극재 가격이 안정화될 전망입니다. 리튬 가격 급락이 촉발한 양극재 ASP 하락이 하반기에 진정될 것으로 예상되기 때문입니다.

이 답변은 다음 증권사 리포트 파일을 인용해서 가져왔습니다 :
- BNK투자증권_포스코퓨처엠_20240409_20240703.txt 파일의 본문 길이 : 548에서 644까지 인용
- BNK투자증권_포스코퓨처엠_20240409_20240703.txt 파일의 본문 길이 : 497에서 547까지 인용
- 포스코퓨처엠_대신증권_20231025_20240703.txt 파일의 본문 길이 : 203에서 377까지 인용
time : 47.9048056602478


In [24]:
# 이전 모델과 토크나이저 언로드
del model
del tokenizer

# 메모리 정리
import gc
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
gc.collect()

489

In [5]:
## 번외 : 양자화 모델을 사용시 

from llama_cpp import Llama

#model_name = "./models/EEVE-Korean-Instruct-10.8B-q4_0.gguf"
model_name = "./models/kiqu-70b.Q2_K.gguf"

llm = Llama(
      model_path=model_name,
      n_gpu_layers= 81, # Uncomment to use GPU acceleration   -1, 0~  
      n_gpu=4,  # 사용할 GPU의 수 설정 (여기서는 4개)
      #seed=1337, # Uncomment to set a specific seed
      n_ctx=4096, # Uncomment to increase the context window
)

llama_model_loader: loaded meta data with 24 key-value pairs and 723 tensors from ./models/kiqu-70b.Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models
llama_model_loader: - kv   2:                       llama.context_length u32              = 32764
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 8192
llama_model_loader: - kv   4:                          llama.block_count u32              = 80
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 28672
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32        

In [6]:
'''
prompt_template = """너는 kiqu-70B라는 한국어에 특화된 언어모델이야. 깔끔하고 자연스럽게 대답해줘!
[INST] 안녕?
[/INST] 안녕하세요! 무엇을 도와드릴까요? 질문이나 궁금한 점이 있다면 언제든지 말씀해주세요.

[INST] {instruction}
[/INST]"""
'''

prompt_template = """너는 kiqu-70B라는 한국어에 특화된 언어모델이야. 깔끔하고 자연스럽게 대답해줘!
[INST] {instruction}
[/INST]"""


#instruction = input('user query =')#"안녕?"
instruction = in_prompt

prompt = prompt_template.format(instruction = instruction)
print(prompt)

너는 kiqu-70B라는 한국어에 특화된 언어모델이야. 깔끔하고 자연스럽게 대답해줘!
[INST] 마지막 질문에 답하려면 다음 문맥을 사용하세요. 답을 모르면 모른다고 말하고 답을 만들어내려고 하지 마세요.
당신은 투자 전문가입니다. 다음 보고서 내용의 일부인 문맥 3개를 기반으로 질문에 성실히 답변해주세요.

## 문맥 : 
양극재 가격 은 리튬 가격에 2분기 후행해서 변동되는데, 리튬 가격이 연초에 바닥을 찍 은 후 2~3월 반등했기 때문에 2Q말~3Q부터는 양극재 가격이 안정화될 전 망이다. 

리튬 가격 급락이 촉발한 양 극재ASP 하락이 하반기에는 진정될 것으로 예상되기 때문이다. 

최근 메탈 가격 하락에 따른 중장기 추정 양극재 ASP 하향 및 실적 변경에 따라 목표주가 조정
4분기까지 이어질 단기 실적 부진이 아쉬우나, 중장기 공급계약으로 바인딩된 물 량 비중이 높은 24년부터 안정적인 실적 성장 가능할 것으로 예상(누적수주합산 얼티엄셀향 35조, SDI향 40조, LGES향 30조). 

질문: 양극재 가격이 안정화될 것이라면 그 근거는 무엇인가?
도움이 되는 답변:
[/INST]


In [7]:
import time
start_time = time.time()

# 입력 텍스트 설정
#prompt = "Hello, how are you?"

# 모델 실행
output = llm(
      prompt, # Prompt
      max_tokens= 4096, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      #stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      stop=['</s>'], #, '## summary', '\n\n'],
      #top_k = 30 ,
      top_p= 0.92 ,
      temperature= 0.85,
      echo= False # Echo the prompt back in the output
)

# 결과 출력
print("Output:", output)
print()
# 실행할 코드 블록 끝 시간 기록
end_time = time.time()

# 실행 시간 계산 및 출력
execution_time = end_time - start_time
print(f"Execution Time: {execution_time} seconds")


llama_print_timings:        load time =    7533.93 ms
llama_print_timings:      sample time =     248.10 ms /  1247 runs   (    0.20 ms per token,  5026.20 tokens per second)
llama_print_timings: prompt eval time =    9401.72 ms /   831 tokens (   11.31 ms per token,    88.39 tokens per second)
llama_print_timings:        eval time =  406446.93 ms /  1246 runs   (  326.20 ms per token,     3.07 tokens per second)
llama_print_timings:       total time =  422682.92 ms /  2077 tokens


Output: {'id': 'cmpl-d4f35172-4be7-4710-9da3-1f75cc6dcbc4', 'object': 'text_completion', 'created': 1720424863, 'model': './models/kiqu-70b.Q2_K.gguf', 'choices': [{'text': ' 양극재 가격의 안정화에 대한 근거는, 리튬 가격의 2~3월에 이른 반등과 관련이 있어요. 리튬 가격이 연초에 바닥을 찍은 후 상승하면서, 양극재 가격 또한 2Q말~3Q부터는 안정화될 전망이 열려요. 이 지출은, 리튬 가격의 상승에 2분기 뒤로 미루어진 변동 특성과 리튬 가격 하락이 촉발한 양 극재ASP의 하락 정도를 감안하메로, 중장기 공급 계약을 통해 유지되는 상태에서 실적의 안정성을 보여주고 있어요. 따라서, 리튬 가격의 변동과 연결된 양극재ASP의 하락이 중단될 것으로 보인다면, 이는 양극재 가격 안정화에 대한 중요한 근거가 될 수 있어요.\n\n이와 관련해서, 최근 메탈 가격 하락과 연계된 중장기 추정 양극재 ASP의 하향 조정도 고려해야 할 요소일 수 있어요. 이러한 상황을 통해, 실질적인 가격 안정화의 여지는 중장기 공급 계약과 관련 업체들의 상활을 포착하는 것이 중요할 거예요. \n\n결국, 리튬 가격의 변동과 연관지어진 양 극재ASP의 하락이 중단될 것으로 보인다면, 이는 양극재 가격 안정화에 대한 강력한 근거가 되어 할 거예요. 하지만, 중장기 공급 계약과 관련 업체들의 상황을 포착할 가능성도 고려해야 하며, 추후에 발생할 수 있는 다른 관련 요소들이 또한 중요한 근거가 될 수 있어요. \n\n양극재 가격의 안정화에 대한 이해를 위해서는, 리튬 가격의 변동과 연관지은 양 극재ASP의 하락 정도, 중장기 공급 계약을 통한 유지되는 상태, 그리고 최근 메탈 가격의 하락과 이에 따른 중장기 추정 양극재 ASP 하향 조정 등을 종합해 보는 것이 필요해요. 이러한 복합적인 요소들을 고려하면, 양극재 가격의 안정화에 대한 근거를 더 명확하게 이해할

In [8]:
print(output['choices'][0]['text'].strip())

양극재 가격의 안정화에 대한 근거는, 리튬 가격의 2~3월에 이른 반등과 관련이 있어요. 리튬 가격이 연초에 바닥을 찍은 후 상승하면서, 양극재 가격 또한 2Q말~3Q부터는 안정화될 전망이 열려요. 이 지출은, 리튬 가격의 상승에 2분기 뒤로 미루어진 변동 특성과 리튬 가격 하락이 촉발한 양 극재ASP의 하락 정도를 감안하메로, 중장기 공급 계약을 통해 유지되는 상태에서 실적의 안정성을 보여주고 있어요. 따라서, 리튬 가격의 변동과 연결된 양극재ASP의 하락이 중단될 것으로 보인다면, 이는 양극재 가격 안정화에 대한 중요한 근거가 될 수 있어요.

이와 관련해서, 최근 메탈 가격 하락과 연계된 중장기 추정 양극재 ASP의 하향 조정도 고려해야 할 요소일 수 있어요. 이러한 상황을 통해, 실질적인 가격 안정화의 여지는 중장기 공급 계약과 관련 업체들의 상활을 포착하는 것이 중요할 거예요. 

결국, 리튬 가격의 변동과 연관지어진 양 극재ASP의 하락이 중단될 것으로 보인다면, 이는 양극재 가격 안정화에 대한 강력한 근거가 되어 할 거예요. 하지만, 중장기 공급 계약과 관련 업체들의 상황을 포착할 가능성도 고려해야 하며, 추후에 발생할 수 있는 다른 관련 요소들이 또한 중요한 근거가 될 수 있어요. 

양극재 가격의 안정화에 대한 이해를 위해서는, 리튬 가격의 변동과 연관지은 양 극재ASP의 하락 정도, 중장기 공급 계약을 통한 유지되는 상태, 그리고 최근 메탈 가격의 하락과 이에 따른 중장기 추정 양극재 ASP 하향 조정 등을 종합해 보는 것이 필요해요. 이러한 복합적인 요소들을 고려하면, 양극재 가격의 안정화에 대한 근거를 더 명확하게 이해할 수 있을 거예요.


In [5]:
def inference_output(prompt):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=False, # 올거나이저 지침사항
        #top_p= 0.92,# 누적 확률을 기준으로 역순으로 단어를 정렬, 지정한 값에 도달하는 순간 멈춤 (단어 선별, 확률 분포의 긴꼬리를 자름 -> 자연스러운 텍스트 생성)
        #top_k=20, # 특별한 이유없으면 1로 지정 , top_p와 다르게 누적 건수를 기준으로 선별 
        #no_repeat_ngram_size=3,
        #temperature= 0.85, # 0.37,
        early_stopping= True,
        #eos_token_id= terminators,  # 종료 토큰 지정 : 2 = </s> ,   46332 =  <|endoftext|>
        ##pad_token_id= tokenizer.eos_token_id,
        #eos_token_id= tokenizer.eos_token_id,
        num_beams=3,
        repetition_penalty=1.05,

        
    )
    response = outputs[0][input_ids.shape[-1]:]

    return tokenizer.decode(response, skip_special_tokens=True)

In [19]:
## 벡터 임베딩 : gpt (필요시 사용, 본 스터디에서는 로컬이 목적)
"""
class MyOpenAIEmbeddingFunction:
    def __init__(self):
        self.model_name = "text-embedding-ada-002"

    def __call__(self, input):
        if isinstance(input, str):
            input = [input]  # Ensure the input is in list form
        
        embeddings = []
        for text in input:
            # API call to get embeddings for each text in the list
            response = openai.Embedding.create(
                model=self.model_name,
                input=text
            )
            # Extract and append the embedding from the response
            embeddings.append(response['data'][0]['embedding'])
        
        return embeddings
# Instantiate the embedding function
embedding_function = MyOpenAIEmbeddingFunction()
"""

In [4]:
## 벡터 임베딩 
from FlagEmbedding import BGEM3FlagModel
import numpy as np

class MyEmbeddingFunction:
    def __init__(self):
        self.model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

    def __call__(self, input):
        if isinstance(input, str):
            input = [input]
        # Encode the input text
        embeddings = self.model.encode(input, 
                                       batch_size=12, 
                                       max_length= 4096)['dense_vecs']
        return embeddings

# Instantiate the embedding function
embedding_function = MyEmbeddingFunction()

bin C:\Users\any\anaconda3\envs\oosij\lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

----------using 4*GPUs----------


In [3]:
## 로컬 모델
### 실험 결과, 금융 로컬모델 중 뛰어난 성능 : 표 데이터 인식 등 
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM
import warnings
warnings.filterwarnings("ignore")

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

#torch.set_default_device('cuda')
device_map = 'auto'

model_id  = "allganize/Llama-3-Alpha-Ko-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    #low_cpu_mem_usage=True,
    #return_dict=True,
    torch_dtype= "auto", # torch.float16,
    device_map=device_map,)

#model.eval()
#model.config.use_cache = (True)


tokenizer = AutoTokenizer.from_pretrained(model_id)
#tokenizer = AutoTokenizer.from_pretrained(model_id, torch_dtype=torch.bfloat16)
#tokenizer.pad_token = tokenizer.eos_token
#tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:

def citation_extarct(results, doc_n):
    check_list = results['documents'][0]
    metas_list = results['metadatas'][0]

    ok_list = []

    for r in range(len(check_list)):
        snippet = check_list[r]
        if doc_n in snippet:
            cit = metas_list[r]['citation']
            p_p = metas_list[r]['source']
        
            ok_list.append([cit, p_p])
    return ok_list

def citation_extract_prompt_add(results, doc_1, doc_2, doc_3):
    line_1, cp_path_1 = citation_extarct(results, doc_1)[0]
    line_2, cp_path_2 = citation_extarct(results, doc_2)[0]
    line_3, cp_path_3 = citation_extarct(results, doc_3)[0]

    citation_template = """이 답변은 다음 증권사 리포트 파일을 인용해서 가져왔습니다 :
- {cp_path_1}파일의 {line_1}에서 인용
- {cp_path_2}파일의 {line_2}에서 인용
- {cp_path_3}파일의 {line_3}에서 인용"""

    citation_prompt = citation_template.format(cp_path_1 = cp_path_1 , cp_path_2 = cp_path_2, cp_path_3 = cp_path_3 ,
                                          line_1 = line_1 , line_2 =  line_2, line_3 = line_3)

    return citation_prompt


# 임의로 만든 인용 추출기 : 구버전 , 함수에서 _v000제거할것
def citation_extract_prompt_add_v000(results, doc_1, doc_2, doc_3): 
    doc_1_c = citation_extarct(results, doc_1)
    doc_2_c =citation_extarct(results, doc_2)
    doc_3_c = citation_extarct(results, doc_3)

    cp_path_1, sp_1, ep_1 = doc_citation_extract(doc_1_c)
    cp_path_2, sp_2, ep_2 = doc_citation_extract(doc_2_c)
    cp_path_3, sp_3, ep_3 = doc_citation_extract(doc_3_c)


    citation_template = """이 답변은 다음 증권사 리포트 파일을 인용해서 가져왔습니다 :
- {citation_pdf_path_1} 파일의 본문 길이 : {start_page_1}에서 {end_page_1}까지 인용
- {citation_pdf_path_2} 파일의 본문 길이 : {start_page_2}에서 {end_page_2}까지 인용
- {citation_pdf_path_3} 파일의 본문 길이 : {start_page_3}에서 {end_page_3}까지 인용"""

    citation_prompt = citation_template.format(citation_pdf_path_1 = cp_path_1 , citation_pdf_path_2 = cp_path_2, citation_pdf_path_3 = cp_path_3 ,
                                          start_page_1 = sp_1 , start_page_2 =  sp_2, start_page_3 = sp_3 ,
                                          end_page_1 = ep_1 , end_page_2 = ep_2, end_page_3 = ep_3 )
    return citation_prompt

def citation_extarct_v000(results, doc_n):
    check_list = results['documents'][0]
    metas_list = results['metadatas'][0]

    ok_list = []

    for r in range(len(check_list)):
        snippet = check_list[r]
        if doc_n in snippet:
            cit = metas_list[r]['citation']
            p_p = metas_list[r]['pdf']
        
            ok_list.append([cit, p_p])
    return ok_list

def doc_citation_extract(doc_n_c):
    dict_str = doc_n_c[0][0]
    # ast.literal_eval을 사용하여 문자열을 딕셔너리로 변환
    dict_obj = ast.literal_eval(dict_str)
    dict_obj['start'], dict_obj['end']

    citation_pdf_path = doc_n_c[0][1].split('\\')[-1] # 경로 

    return citation_pdf_path, dict_obj['start'], dict_obj['end']

def inference_output(prompt):
    terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    #input_ids = tokenizer.apply_chat_template(messages,add_generation_prompt=True,return_tensors="pt").to(model.device)
    # 인퍼런스 커스텀
    #inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False).to("cuda") # return_token_type_ids=False
    # peft 일시, **tokenizer(prompt, return_tensors="pt", add_special_tokens=False) 아래 inputs['input_ids'] 교체 , 위는 삭제 
    output = model.generate(
        **tokenizer(prompt, return_tensors="pt", add_special_tokens=False),
        max_new_tokens = 1024, # 200  256 512
        do_sample= True, # temperature 매개변수는 top_p 및 top_k와 함께 do_sample=True일 때만 활성화
        top_p= 0.92,# 누적 확률을 기준으로 역순으로 단어를 정렬, 지정한 값에 도달하는 순간 멈춤 (단어 선별, 확률 분포의 긴꼬리를 자름 -> 자연스러운 텍스트 생성)
        top_k=20, # 특별한 이유없으면 1로 지정 , top_p와 다르게 누적 건수를 기준으로 선별 
        no_repeat_ngram_size=3,
        temperature= 0.3, # 0.37,
        early_stopping= True,
        eos_token_id=terminators,  # 종료 토큰 지정 : 2 = </s> ,   46332 =  <|endoftext|>
        repetition_penalty=1.2,
        #pad_token_id= tokenizer.eos_token_id,
        #eos_token_id= tokenizer.eos_token_id,
        num_beams=3,
    )
    output = output[0].to("cpu")
    return tokenizer.decode(output)

# Document 클래스 정의
class Document:
    def __init__(self, page_content, metadata):
        self.page_content = page_content
        self.metadata = metadata

    def __str__(self):
        return f"Document(page_content={self.page_content}, metadata={self.metadata})"

# 텍스트를 일정한 크기로 나누고 오버랩하는 클래스
class CharacterTextSplitter:
    def __init__(self, chunk_size, chunk_overlap, separator=''):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.separator = separator

    def split_text(self, text):
        chunks = []
        start = 0
        while start < len(text):
            end = min(start + self.chunk_size, len(text))
            if self.separator:
                end = text.find(self.separator, start, end)
                if end == -1:
                    end = min(start + self.chunk_size, len(text))
                else:
                    end += len(self.separator)
            chunk = text[start:end]
            chunks.append((chunk, start, end - 1))
            start += self.chunk_size - self.chunk_overlap
        return chunks

    def split_documents(self, documents):
        split_docs = []
        for doc in documents:
            text_chunks = self.split_text(doc.page_content)
            for chunk, start_p, end_p in text_chunks:
                # 메타데이터에 citation 정보 추가
                metadata_with_citation = {**doc.metadata, 'citation': {'start_p': start_p, 'end_p': end_p}}
                split_docs.append(Document(page_content=chunk, metadata=metadata_with_citation))
        return split_docs


def timestamp_convert(documents): # datetime string
    key_add = 'timestamp'
    for doc in documents:
        datetime_object = doc.metadata['wdate']
        if isinstance(datetime_object, str):
            # wdate가 문자열인 경우, datetime 객체로 변환
            datetime_object = datetime.strptime(datetime_object, "%Y-%m-%d %H:%M:%S")
        timestamp = int(datetime_object.timestamp())
        doc.metadata[key_add] = timestamp
        doc.metadata['wdate'] = str(datetime_object)  # datetime 객체를 str로 변환
    return documents

# reranker model load
def reranker_model(query_text, doc_n):
    def exp_normalize(x):
        b = x.max()
        y = np.exp(x - b)
        return y / y.sum()
    
    rmodel_path = 'Dongjin-kr/ko-reranker'

    rtokenizer = AutoTokenizer.from_pretrained(rmodel_path)
    rmodel = AutoModelForSequenceClassification.from_pretrained(rmodel_path)
    rmodel.eval()

    pairs = []

    for i in range(len(doc_n)):
        docs = doc_n[i]
        query_doc = [query_text, docs ]
        pairs.append(query_doc)

    with torch.no_grad():
        inputs = rtokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = rmodel(**inputs, return_dict=True).logits.view(-1, ).float()
        scores = exp_normalize(scores.numpy())
        #print (f'first: {scores[0]}, second: {scores[1]}')
    sorted_indices = np.argsort(-scores)
    sorted_values = scores[sorted_indices]

    index_numbers = sorted_indices[:3]

    sent_list = []

    for d in range(len(index_numbers)):
        index_n = index_numbers[d]
        sent = doc_n[index_n]
        #print(sent)
        sent_list.append(sent)
    return sent_list

## 함수 추가 

# 패턴이 동일하면 할 날짜 
def file_date_format(file_path):
    # 정규 표현식을 사용하여 날짜 추출
    date_string = re.search(r'\d{6}', file_path).group()

    # 문자열을 datetime 객체로 변환
    date_object = datetime.strptime(date_string, '%Y%m%d')
    
    return date_object


# 표 데이터 제거
def remove_table_data(text):
    # 숫자와 기호로 이루어진 표 패턴 탐지 및 제거
    table_pattern = re.compile(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?(?:[\%\원억십조천백]*)?\s*)+', re.MULTILINE)
    cleaned_text = re.sub(table_pattern, '', text)
    
    # '표' 또는 'Fig'가 들어간 줄 제거
    cleaned_text = re.sub(r'표\d+.*|Fig\.\s*\d+.*', '', cleaned_text)
    
    # 추가적으로 다중 공백, 특수 문자 정리
    cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text)  # 다중 공백을 한 줄로 정리
    cleaned_text = re.sub(r'[^\S\r\n]+', ' ', cleaned_text)  # 다중 공백을 한 칸 공백으로 정리
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # 연속된 공백을 한 칸 공백으로 정리
    cleaned_text = re.sub(r'\n+', '\n', cleaned_text)  # 연속된 줄 바꿈을 한 줄로 정리
    cleaned_text = re.sub(r'[ ]*표[ ]*\d+', '', cleaned_text)  # '표'와 숫자가 포함된 문자열 제거
    cleaned_text = re.sub(r'Fig[ ]*\d+', '', cleaned_text)  # 'Fig'와 숫자가 포함된 문자열 제거
    cleaned_text = remove_specific_pattern(cleaned_text)
    

# 특정 패턴 제거 함수
def remove_specific_pattern(text):
    # 텍스트 내의 특정 패턴을 정의하고 제거
    specific_pattern = re.compile(r'\n\(천 원 \)\n\n\n\n\n\n\n\n\n\(십 억 원 \)\n\n\n\n\n\n\(\)\n\(\)\n\(\)\n\n\n\n\n\n\n주\n\n\n\n가\n\.X\n\n\n\n영\n\n\n\n업\n\n\n\n현\n\n\n\n금\n\n\.X\n\.X\n\n흐 름\n\n\n\n\n\n\.X\n\.X\n\n\n\n\n\n\n\(천 원\n\n\n\n\n\n\n\n\(십 억\n,\n,\n,\n,\n\n\)\n\n원\n\n\n\n\n\n\n\n\)\n\n\n\n\n\n\n\n\n\n\n주\n\n가\nX\n\n\n\n\nX\nX\n\nC a p e x\n\n\n\n\n\n\n\n\n\n\n\nX\nX\n\n\n\n\n\n\n\(십억원\) \(십억원\)\nFree Cash Flow 순차입금\n\n\n\n\n\(\) \n\n\(\) \n\n\(\)\n\n\(\) \(\)\n\n재무상태표 포괄손익계산서\n \n \n \n \n 률 \n비 \n \n \n 률 \n \n ---\n ----\n -\n비 기타 --\n -\n 률 -\n -\n \n \n \n 률 \n \n -\n현금흐름표 주요투자지표\n \n --\n \n \n \n -----\n ----\n감소 ----\n감소 -----\n증가 EV/ \n -----\n -----\n증가 -----증가율 \n감소 증가율 -\n순감 ----\n --\n \n \n -----/자기자본 \n -\n \n ------\n', re.MULTILINE)
    cleaned_text = re.sub(specific_pattern, '', text)
    
    return cleaned_text
    
def clean_text_by_pdf(text):
    # 정규식을 이용해 표를 제거
    clean_text = re.sub(r"\n주.*\n", "\n", text)  # '주:'로 시작하는 줄 제거
    clean_text = re.sub(r"\n\s*PER.*\n", "\n", clean_text)  # 'PER'로 시작하는 줄 제거
    clean_text = re.sub(r"\d{4} \d{4} \d{4} \d{4}", "", clean_text)  # 연도별 데이터 줄 제거
    #clean_text = re.sub(r'(?<=[가-힣])\\n(?=[가-힣])', '', clean_text)
    #clean_text = re.sub(r'(?<=[가-힣])\n(?=[가-힣])', '', clean_text)
    clean_text = clean_text.replace('\\n\\n','')

    return clean_text


def prompt_clean_pdf_content(pdf_text, model_name):
    p_template = """pdf 파일의 내용을 텍스트 추출기를 통해 추출한 내용을 전처리 하십시오. 다음 지침을 충실히 따르십시오.
    - "다음은 테이블과 이미지 관련 내용을 제외한 텍스트 내용입니다:" 와 같은 서두는 하지말아야 합니다.
    - 매출전표, 재무재표와 같은 표로 구성된 텍스트는 제외해주세요.
    -  ■■■■■ 이렇게 붙은 텍스트 행도 표로 간주하고 제외해주세요.

    ### 삭제해야할 숫자 포함 표 텍스트 행 :
    매출액 3,302 4,760 4,619 6,040 8,457 유동자산 2,038 2,412 2,400 3,516 2,825

    매출원가 2,967 4,503 4,215 5,527 7,738 현금및현금성자산 281 390 310 1,284 118
        
    매출총이익 335 257 405 513 719 매출채권 및 기타채권 292 770 778 855 1,069

    {pdf_text}
    """

    prompt = p_template.format(pdf_text = pdf_text)

    result = get_completion(prompt, model_name, temperature=0, verbose=False)
    return result

#pdf_text
#text  = clean_text_by_pdf(pdf_text)

# 결과 출력
#print(text)

def prompt_template_news(ticker, title, body):
    prompt_template = """Based on the following news article, please come up with one of the most important questions that average investors would ask.
only answer is query. 

# news article : 
- ticker : 
{ticker}

- article title : 
{title}

- article content : 
{body}


# query generate : """
    prompt_insert = prompt_template.format(ticker = ticker, title = title, body = body)
    
    return prompt_insert


# 텍스트를 임베딩으로 변환하는 함수 정의
def embed_text(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.squeeze().tolist()

# error, retry 추가
def get_completion(prompt, model, temperature=0, verbose=False):
    messages = [{"role": "user", "content": prompt}]
    
    time_start = time.time()
    retry_count = 3
    for i in range(0, retry_count):
        while True:
            try:    
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=messages,
                    temperature=temperature, # this is the degree of randomness of the model's output
                )
                answer = response['choices'][0]['message']['content'].strip()                
                tokens = response.usage.total_tokens                
                
                
                time_end = time.time()
                
                if verbose:
                    print('prompt: %s | token: %d | %.1fsec\nanwer : %s'%(prompt, tokens, (time_end - time_start), answer))
                return answer
            
            except Exception as error:
                print(f"API Error: {error}")
                print(f"Retrying {i+1} time(s) in 4 seconds...")
                
                if i+1 == retry_count:
                    return prompt, None, None
                time.sleep(4)
                continue
                
def target_date_select(target_date):
    if target_date.find(',') >= 0 :
        td_list = target_date.split(',')
    else:
        td_list = [target_date.strip()]
    return td_list 

def target_datetime_extract(date_range_string, data):
    date_range_string = date_range_string.strip()
    start_date_str, end_date_str = date_range_string.split("~")

    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

    target_date_data = [
        item for item in data if start_date.date() <= item[3].date() < end_date.date()
    ]
    return target_date_data

def oracle_news_load(code):
    sql = """
    SELECT A.SN, TITLE, CONTENT, WDATE, FILENAME FROM

    (SELECT * FROM NEWS_VIEW WHERE CODES = '"""+code+"""' AND SOURCE ='10' AND  ROWNUM <= 10000) A, 

    (SELECT * FROM NEWS_CONT)B

    WHERE A.SN = B.SN ORDER BY SN DESC 
    
    """

    cursor = conn.cursor()
    cursor.execute(sql)
    
    # 결과 레코드를 차례대로 리스트에 저장
    result_list = []
    for row in cursor:
        result_list.append(row)
        
    return result_list

## 문장 분리기 : 31 이하 단어 수는 삭제 (불필요한 짧은 문장 제거)
def body_split_token(text):
    #kkma = Kkma()
    #token_sentence = kkma.sentences(text)
    token_sentence =  split_sentences(text)
    doc_token_list = []

    for t in range(len(token_sentence)): 
        doc_token = token_sentence[t]
        doc_token = re.sub('&nbsp-','', doc_token)
        if len(doc_token) <= 31:
            continue

        doc_token_list.append(doc_token)
        
    return doc_token_list


def oracle_sql_select(code):
    sql = """
    SELECT A.SN, TITLE, CONTENT FROM

    (SELECT * FROM NEWS_VIEW WHERE CODES = '"""+code+"""' AND SOURCE ='10' AND  ROWNUM <= 1000) A, 

    (SELECT * FROM NEWS_CONT)B

    WHERE A.SN = B.SN ORDER BY SN DESC 

    """
    cursor = conn.cursor()
    cursor.execute(sql)
    
    # 결과 레코드를 차례대로 리스트에 저장
    result_list = []
    for row in cursor:
        result_list.append(row)
        
    return result_list

def sql_output_data(result_list, code):
    news_list = []

    for r in range(len(result_list)):
        index_number = result_list[r][0]
        title = result_list[r][1].strip()
        body_origin = result_list[r][2].read()
        soup = BeautifulSoup(body_origin, 'html.parser')
        text_body = soup.find_all('p')
        if len(text_body) <= 0:
            body = body_origin
        else:
            # BeautifulSoup을 사용하여 HTML 파싱
            body = body_html_del(body_origin)
            # BeautifulSoup을 사용하여 HTML 파싱
        http_only_check = http_url_search(body)
        if http_only_check == True:  # 링크들만 있으면 스킵
            continue
        if len(body) ==0:
            continue
        ndata = [index_number, code, title, body]
        news_list.append(ndata)
        
    return news_list

## 함수 만들기 

def body_html_del(body_origin):
    soup = BeautifulSoup(body_origin, 'html.parser')
    text_body = soup.find_all('p')
    body_p = ''
    for b in range(len(text_body)):
        tbody = text_body[b].text
        tbody = tbody.replace('\xa0','')
        body_kor_check = contains_korean(tbody)

        if body_kor_check == False:  # 링크들만 있으면 스킵
            continue
        if b == 0:
            body_p = tbody
        else:
            body_p = body_p +' '+ tbody
        
    return body_p

# 본문 http url 링크만 있는 경우 탐지 
def http_url_search(text):
    pattern = r'http\S+'
    find_url = re.compile(pattern)
    
    return  bool(find_url.search(text))


### 여기는 전처리 후보인데, 현재는 사용하지 않음 
## 본문 전처리 1
def body_preprocessing(body):
    # 정규식을 사용해 불필요한 태그 제거
    text = re.sub('<.*?>', '', body)
    # [숫자] 패턴 제거
    text = re.sub(r'\[\d+\]', '', text)
    # [문자열] 패턴 제거
    text = re.sub(r'\[[^\]]+\]', '', text)
    return text

## 한글 체크하기 
def contains_korean(text):
    korean = re.compile('[ㄱ-ㅣ가-힣]+')
    return bool(korean.search(text))

# 본문 http url 링크 삭제하기 
def text_http_del(text):
    pattern = r'http\S+'
    text_without_urls = re.sub(pattern, '', text)
    
    return text_without_urls


def extract_text_from_html(html):
    # HTML 태그 제거
    html = re.sub(r'<[^>]*>', '', html)
    
    # CSS 스타일 태그 제거
    html = re.sub(r'<style.*>.*<\/style>', '', html, flags=re.DOTALL)
    
    # 불필요한 공백 문자 제거
    html = re.sub(r'\s+', ' ', html)
    
    # 텍스트 추출
    text = html.strip()
    
    return text

def extract_korean(text):
    korean_pattern = re.compile('[^ㄱ-ㅣ가-힣]+')
    korean_text = korean_pattern.sub(' ', text)
    return korean_text

def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

# 뉴스 본문 정규표현식 전처리 모듈 : 기자 나누기 
def newsbody_preprocessing(body):
    text_filter = re.compile('[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z0-9&,\.\?\!\"\'-()\[\]\{\}]')
    email_pattern = re.compile('^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$')
    reporter_pattern = re.compile('[가-힣]{2,4},\s?[가-힣]{2,4}\s?기자|([가-힣]{2,4})\s?기자')
    doublespace_pattern = re.compile('\s+')
    
    text = email_pattern.sub(' ', body)
    reporter = reporter_pattern.search(text)
    if reporter != None:
        reporter = reporter.group(0)
    # text = reporter_pattern.sub(' ', text)
    text = text_filter.sub(' ', text)
    text = doublespace_pattern.sub(' ', text).strip()
    text = text.split(reporter)
    text = text[0]
    return text, reporter

## pdf 로더 
def extract_text_images_tables(pdf_path):
    all_text = ""
    images = []
    tables = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extract text
            all_text += page.extract_text() + "\n"

            # Extract images
            for image in page.images:
                images.append(image)

            # Extract tables
            for table in page.extract_tables():
                tables.append(table)
    
    return all_text, images, tables


def extract_text_without_tables_images(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Extracting raw text from the page
            page_text = page.extract_text()
            
            if page_text is None:
                continue
            
            # Extracting tables from the page
            tables = page.extract_tables()

            # Filter out text that belongs to tables
            if tables:
                for table in tables:
                    for row in table:
                        for cell in row:
                            if cell is not None:
                                page_text = page_text.replace(cell, '')

            # Adding the cleaned page text to the final text
            text += page_text + "\n"

    return text

def pdf_loader(pdf_path):
    #pdf_path ='./data/삼성전자샘플_005930.pdf'
    # Extract text, images, and tables from the PDF
    extracted_text = extract_text_without_tables_images(pdf_path)
    # image / table can't

    # Print the extracted text
    pdf_text = repr(extracted_text)
    
    return pdf_text

def get_pdf_paths(folder_path):

    pdf_paths = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".pdf"):
                pdf_paths.append(os.path.join(root, file))
    return pdf_paths

def pdf_loader_by_folder(data_folder_path):
    pdf_files = get_pdf_paths(data_folder_path)
                         
    page_list = []

    for p in range(len(pdf_files)):
        pdf_path = pdf_files[p]
        page = pdf_loader(pdf_path)
        page_list.append(page)
    return page_list, pdf_files